<a href="https://colab.research.google.com/github/elephant-xyz/photo-meta-data-notebook/blob/main/PhotoMedtaData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📥 Step 1: Upload the `.env` File

This notebook requires a `.env` file that contains your API keys and credentials.  
It will be used to securely load the following environment variables:

| Variable Name           | Purpose                     |
|-------------------------|-----------------------------|
| `OPENAI_API_KEY`        | Access to OpenAI API        |
| `AWS_ACCESS_KEY_ID`     | AWS access key              |
| `AWS_SECRET_ACCESS_KEY` | AWS secret access key       |
| `S3_BUCKET_NAME`        | AWS secret access key       |
| `IMAGES_DIR`            | Images                      |



- Click the **folder icon** 📂 in the left sidebar to open the file browser.
- Then click the **"Upload"** button and choose your `.env` file.

example of .env file

OPENAI_API_KEY=sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXX
AWS_ACCESS_KEY_ID=XXXXXX
AWS_SECRET_ACCESS_KEY=XXXXXX
S3_BUCKET_NAME=your-s3-bucket-name-here
IMAGES_DIR=images


## Step 2: Upload `seed.csv`

Upload the `seed.csv` file to the `/content/` directory. This file contains image-level seed data such as metadata, image references, or tagging prompts.

## Step 3: Upload `upload_results.json`

Upload the `upload_results.json` file to the `/content/` directory.

> 📌 **Important**: You must generate this file by running **Step 2** of the following Colab notebook:  
> 👉 [Upload Images Notebook – Step 2](https://colab.research.google.com/drive/14tSNSP8Pe-mY4VwX9JhXgfyOvzmN3kC0?usp=sharing#scrollTo=OFKp4E49651Z)

After running Step 2 in that notebook, download the `upload_results.json` file and upload it to this notebook's `/content/` directory.



test

In [27]:
# 1. Install the package
!pip install --force-reinstall --no-cache-dir git+https://github.com/elephant-xyz/photo-meta-data-ai.git > /content/install_log.txt 2>&1


# 3. Set up folders
!folder-setup

# 4. Verify folders were created
!ls -la /content/images/

✓ Loaded environment from .env
✓ Loaded seed file: seed.csv
  - Rows: 2
  - Columns: ['parcel_id', 'Address', 'method', 'headers', 'url', 'multiValueQueryString', 'body', 'json', 'source_identifier', 'County']
✓ Created root folder: /content/images
Found 2 unique parcel IDs
✓ Created folder: 30434108090030050
✓ Created folder: 52434205310037080
total 24
drwxr-xr-x 4 root root  4096 Jul 22 18:14 .
drwxr-xr-x 1 root root  4096 Jul 22 18:18 ..
drwxr-xr-x 2 root root 12288 Jul 22 18:24 30434108090030050
drwxr-xr-x 2 root root  4096 Jul 22 18:23 52434205310037080


## Step 4: Upload images with Parcel ID Subfolders

Place all image files related to that parcel inside its corresponding folder under IMAGE_FOLDER_NAME=images






## Step 5: Setup Environmet to process images

Place all image files related to that parcel inside its corresponding folder under IMAGE_FOLDER_NAME=images


In [28]:
!bucket-manager

## Step 6

Categorizing the photos


In [29]:
!photo-categorizer


📊 COMPREHENSIVE CATEGORIZATION SUMMARY

🏠 TOTAL PROPERTIES PROCESSED: 2
🖼️  TOTAL IMAGES: 203
✅ TOTAL CATEGORIZED: 203
📈 SUCCESS RATE: 100.0%

📁 OVERALL CATEGORY BREAKDOWN:
   exterior: 68 images
   kitchen: 42 images
   living_room: 40 images
   bedroom: 20 images
   other: 14 images
   closet: 6 images
   garage: 4 images
   pool: 3 images
   laundry: 2 images
   bathroom: 2 images
   dining_room: 2 images

🏠 PROPERTY-BY-PROPERTY BREAKDOWN:
--------------------------------------------------------------------------------

📍 Property: 30434108090030050
   Address: 1605 S US HIGHWAY 1 3E,PALM BEACH GARDENS
   Total Images: 153
   Categorized: 153
   Success Rate: 100.0%
   Categories:
     • exterior: 60 images
     • living_room: 30 images
     • kitchen: 24 images
     • other: 14 images
     • bedroom: 12 images
     • closet: 4 images
     • garage: 4 images
     • pool: 3 images
     • laundry: 2 images

📍 Property: 52434205310037080
   Address: 2558 GARDENS PKWY,JUPITER
   Total 

!ai-analyzer
# New Section

AI ANALYSIS


In [ ]:
!pip install --force-reinstall --no-cache-dir git+https://github.com/elephant-xyz/photo-meta-data-ai.git > /content/install_log.txt 2>&1

!ai-analyzer --local-folders


✓ AWS S3 authentication successful
    [→] Processing batch 1 (5 images)
[DEBUG] Starting OpenAI API call for 5 images (attempt 1/3)
[DEBUG] Processing 5 images for API call
    [→] Processing batch 2 (5 images)
[DEBUG] Processing image 1/5: images/30434108090030050/living_room/011-2558GardensPkwy-PalmBeachGardens-FULL.jpg
[DEBUG] Starting OpenAI API call for 5 images (attempt 1/3)
    [→] Processing batch 3 (4 images)
[DEBUG] Processing 5 images for API call
[DEBUG] Starting OpenAI API call for 4 images (attempt 1/3)
[DEBUG] Processing image 1/5: images/30434108090030050/living_room/027-2558GardensPkwy-PalmBeachGardens-FULL.jpg
[DEBUG] Processing 4 images for API call
[DEBUG] Processing image 1/4: images/30434108090030050/living_room/017-2558GardensPkwy-PalmBeachGardens-FULL.jpg
[DEBUG] Successfully encoded image 1
[DEBUG] Processing image 2/5: images/30434108090030050/living_room/015-2558GardensPkwy-PalmBeachGardens-FULL.jpg
[DEBUG] Successfully encoded image 1
[DEBUG] Successfully e